Notebook para analisar os dados da base Bolsa Família

In [2]:
# Importando a sessão do Spark que se comunica com o cluster
from pyspark.sql import SparkSession

In [3]:
#Instanciando a sessão spark
spark = SparkSession.builder.appName("Bolsa Familia").getOrCreate()

In [13]:
caminho = "./data/202401_NovoBolsaFamilia.csv"

df = spark.read.csv(caminho, 
                    header=True,
                    sep=';',
                    encoding='ISO-8859-1')

In [14]:
df.show()

+---------------+--------------+---+----------------------+--------------------+--------------+--------------+--------------------+-------------+
|MÊS COMPETÊNCIA|MÊS REFERÊNCIA| UF|CÓDIGO MUNICÍPIO SIAFI|      NOME MUNICÍPIO|CPF FAVORECIDO|NIS FAVORECIDO|     NOME FAVORECIDO|VALOR PARCELA|
+---------------+--------------+---+----------------------+--------------------+--------------+--------------+--------------------+-------------+
|         202401|        202303| AC|                  0139|          RIO BRANCO|***.511.972-**|   16291115205|JANAINA FERREIRA ...|       800,00|
|         202401|        202303| BA|                  3329|             ANGUERA|***.909.745-**|   20474429886|TACIANE MARQUES D...|       750,00|
|         202401|        202303| BA|                  3731|         MONTE SANTO|***.872.338-**|   20195376298|MAILSON DOS SANTO...|       650,00|
|         202401|        202303| BA|                  3761|         NOVA VICOSA|***.665.905-**|   12723589082|IRENI RICARDO 

In [15]:
df.printSchema()

root
 |-- MÊS COMPETÊNCIA: string (nullable = true)
 |-- MÊS REFERÊNCIA: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- CÓDIGO MUNICÍPIO SIAFI: string (nullable = true)
 |-- NOME MUNICÍPIO: string (nullable = true)
 |-- CPF FAVORECIDO: string (nullable = true)
 |-- NIS FAVORECIDO: string (nullable = true)
 |-- NOME FAVORECIDO: string (nullable = true)
 |-- VALOR PARCELA: string (nullable = true)

